In [5]:
import pandas as pd
import numpy as np
import re
from wordcloud import WordCloud, STOPWORDS 
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer  
import pickle

In [6]:
def Preprocessing_lemmatization_spacy(df,column_name):
    # Email Subject column
    df[column_name] = df[column_name].apply(lambda x: re.sub('[^A-Za-z]'," ",x))
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    df[column_name] = df[column_name].apply(lambda x: _RE_COMBINE_WHITESPACE.sub(" ",x).strip())
    stopwords = set(STOPWORDS) 
    nlp = spacy.load("en_core_web_sm")
    df[column_name] = df[column_name].apply(lambda x : x.lower())
    df[column_name] = df[column_name].apply(lambda x : " ".join(word for word in x.split() if len(word) > 2))
    df[column_name] = df[column_name].apply(lambda x : " ".join([word for word in x.split() if word not in stopwords]))
    df[column_name] = df[column_name].apply(lambda x : " ".join(token.lemma_ for token in nlp(x))) 
    return df[column_name]

## Testing

In [7]:

def prediction_single_observation(tower_name,application_name,email_subject,email_body):
    
    # input
    tower_nm = str(tower_name)
    app_nm = str(application_name)
    email_sb = str(email_subject)
    email_bdy = ""
    
    #preprocessing
    combined = tower_nm + " " + app_nm + " " + email_sb + " " + email_bdy
    
    combined = re.sub('[^A-Za-z]'," ",combined)
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    combined = _RE_COMBINE_WHITESPACE.sub(" ", combined).strip()
    stopwords = set(STOPWORDS) 
    nlp = spacy.load("en_core_web_sm")
    combined = combined.lower()
    combined = " ".join(word for word in combined.split() if len(word) > 2)
    combined = " ".join([word for word in combined.split() if word not in stopwords])
    combined = " ".join(token.lemma_ for token in nlp(combined))
    
    # Feature Extraction
    # load the feature extarctor
    tf_idf = pickle.load(open("tfidf.pkl", 'rb'))
    
    combined = pd.Series(combined)
    test_data = tf_idf.transform(combined)
    
    # load the model from disk
    loaded_model = pickle.load(open("random_forest_tfidf.pkl", 'rb'))
    result = loaded_model.predict(test_data)
    
    if result[0] == 0:
        print("Actionable")
        
    else:
        print("Non Actionable")

        
        
    

def prediction_data(path):
    
    # Reading
    try:
        data = pd.read_excel(path) 
    except:
        data = pd.read_csv(path) 
    
    
    #preprocessing
    data['combined'] = data.apply(lambda x : f"{x[0]} {x[1]} {x[2]}",axis = 1)
    data["combined"] = Preprocessing_lemmatization_spacy(data, "combined")
    
    
    # Feature Extraction
    # load the feature extarctor
    tf_idf = pickle.load(open("tfidf.pkl", 'rb'))
    test_data = tf_idf.transform(data["combined"])
    
    # load the model 
    loaded_model = pickle.load(open("random_forest_tfidf.pkl", 'rb'))
    result = loaded_model.predict(test_data)
    #print(result)
    
    #concat prediction
    data["Flag"] = result
    data["Flag"] = data["Flag"].map({0 : "Actionable" , 1 : "Non Actionable"})
    
    data.to_csv("result_data.csv")
    print("check data in ypur current directory")

## Checking

In [8]:
prediction_single_observation('Core Pharmacy' ,'PharmRDS-Accredo RealTime Messages','error alert please solve',"")

Actionable


In [9]:
prediction_data("Book1.xlsx")

check data in ypur current directory
